# Retrieving Data from the Fantasy Premier League API

### Getting Player IDs for 2023-2024 Season

In [2]:
# Import Libraries
import requests, json
from pprint import pprint
import pandas as pd

In [3]:
base_url = 'https://fantasy.premierleague.com/api/' 
gen = requests.get(base_url+'bootstrap-static/').json() 

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# Looking at Structure of Elements
players = gen['elements']

In [6]:
# Want id, first_name, second_name, team
players_df = pd.DataFrame(players)
players_df = players_df[['id', 'first_name', 'second_name', 'team']]

### Getting Team IDs

In [7]:
teams_df = pd.DataFrame(gen['teams'])
teams_df = teams_df[['id','name']]

### Joining Teams to Players

In [8]:
df = pd.merge(players_df, teams_df, left_on = 'team', right_on= 'id', how = 'left')
df.drop(columns = ['team', 'id_y'], inplace=True)
df.rename(columns={'id_x': 'player_id', 'second_name' : 'last_name'}, inplace=True)

### Retrieving Player Data for 2023-2024 Season

In [9]:
import pandas as pd
import requests

# Initialize Data Frame to store all Gameweek data for all players
player_data_df = pd.DataFrame()

# Loops through all players that have played this season
for i in range(len(df)):
    base_url = 'https://fantasy.premierleague.com/api/' 
    player_id = i
    player_iter = requests.get(base_url + f"element-summary/{player_id}/").json()
    
    # Print the JSON response to inspect its structure
    #print(f"Player {i} JSON response:", player_iter)
    
    # Check if 'history' key exists in the JSON response
    if 'history' in player_iter:
        player_iter = pd.DataFrame(player_iter['history'])
        # Select relevant columns
        player_iter = player_iter[['opponent_team','total_points','was_home','team_h_score','team_a_score',
                                     'round','minutes','goals_scored','assists','clean_sheets','goals_conceded',
                                     'own_goals','penalties_saved','penalties_missed','yellow_cards','red_cards',
                                     'saves','bonus','bps', 'starts','expected_goals','expected_assists',
                                     'expected_goal_involvements','expected_goals_conceded','value','selected',
                                     'transfers_in','transfers_out']]
        # Add Player ID
        player_iter['player_id'] = i
        # Rename Columns
        player_iter.rename(columns = {'round':'gameweek'}, inplace = True)
        # Changing Data Types
        player_iter = player_iter.astype({'expected_goals':float, 'expected_assists':float,'expected_goal_involvements':float,
                                                'expected_goals_conceded':float})
        # Concatenate each player's data
        player_data_df = pd.concat([player_data_df, player_iter], ignore_index=True)
    else:
        pass

In [10]:
# Create Copy
player_final = player_data_df

In [11]:
# Merge Player Data
player_final = pd.merge(player_final,df,'left', on='player_id')

# Merge Opponent Team Data
player_final = pd.merge(player_final, teams_df, 'left', left_on='opponent_team', right_on='id')

In [12]:
# Rename Columns
player_final.rename(columns={'name_x': 'player_team_name', 'name_y' : 'opponent_team_name'}, inplace=True)

In [13]:
player_final['player_name'] = player_final['first_name'] + ' ' + player_final['last_name']
player_final.drop(columns=['first_name','last_name','id','opponent_team'],inplace=True)

In [14]:
# Change Order of Columns
# Reordered Columns
reordered_columns = ['player_id','player_name','player_team_name','opponent_team_name']
# Define the remaining columns
remaining_columns = [col for col in player_final.columns if col not in reordered_columns]
# Reorder columns and append remaining columns
player_final = player_final[reordered_columns + remaining_columns]

In [15]:
# Divide Value by 10 (Get to millions)
player_final['value'] = player_final['value']/10 

In [19]:
# Send to csv
player_final.to_csv('/Users/michaelsabor/Library/CloudStorage/OneDrive-Personal/GitHub/personal-repo/FPL Project/Data/player_data_202324', index=False)